In [147]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [148]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [149]:
data = pd.read_excel('data/data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [150]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [151]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [152]:
x.drop('lat', axis = 1, inplace = True)

In [153]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [154]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [155]:
from sklearn.feature_selection import RFE

In [156]:
#выделим факторы и целевой признак
y = data['price']
x = data.drop(columns='price')
#удалим избыточный признак
x.drop('lat', axis = 1, inplace = True)

In [157]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [158]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'weather'], dtype=object)

In [159]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'long', 'weather'], dtype='object')

In [160]:
selector.ranking_

array([1, 1, 3, 2, 1])

Обучим модель линейной регрессии на трех наиболее значимых признаках по методу RFE

In [161]:
X_train_rfe = X_train[['year', 'cylinders', 'weather']]
X_test_rfe = X_test[['year', 'cylinders', 'weather']]

lr_rfe = LinearRegression()
lr_rfe.fit(X_train_rfe, y_train)

#Делаем предсказание для тренировочной выборки:
y_train_predict_rfe = lr_rfe.predict(X_train_rfe)
#Делаем предсказание для тестовой выборки:
y_test_predict_rfe = lr_rfe.predict(X_test_rfe)

#Выведем коэффициент детерминации для тренировочной и тестовой выборок
from sklearn.metrics import r2_score
print("Train R^2: {:.3f}".format(r2_score(y_train, y_train_predict_rfe)))
print("Test R^2: {:.3f}".format(r2_score(y_test, y_test_predict_rfe)))

#выведем среднюю абсолютную ошибку
mae = mean_absolute_error(y_test, y_test_predict_rfe)
print('MAE: %.3f' % mae)

Train R^2: 0.479
Test R^2: 0.573
MAE: 5113.445


###  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [162]:
from sklearn.feature_selection import SelectKBest, f_regression

In [163]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [ ]:
Обучим модель линейной регрессии на трех наиболее значимых признаках по методу выбора признаков на основе фильтров

In [165]:
X_train_filter = X_train[['year', 'cylinders', 'odometer']]
X_test_filter = X_test[['year', 'cylinders', 'odometer']]

lr_filter = LinearRegression()
lr_filter.fit(X_train_filter, y_train)

#Делаем предсказание для тренировочной выборки:
y_train_predict_filter = lr_filter.predict(X_train_filter)
#Делаем предсказание для тестовой выборки:
y_test_predict_filter = lr_filter.predict(X_test_filter)

#Выведем коэффициент детерминации для тренировочной и тестовой выборок
from sklearn.metrics import r2_score
print("Train R^2: {:.3f}".format(r2_score(y_train, y_train_predict_filter)))
print("Test R^2: {:.3f}".format(r2_score(y_test, y_test_predict_filter)))

#выведем среднюю абсолютную ошибку
mae = mean_absolute_error(y_test, y_test_predict_filter)
print('MAE: %.3f' % mae)

Train R^2: 0.511
Test R^2: 0.602
MAE: 4708.946


Исходя из проведенной работы можно сделать вывод, что для данной выборки метод на основе фильтров для трех признаков дал лучшие результаты, чем метод RFE (так же для 3 признаков): коэффициенты детерминации у него оказались выше, чем у метода RFE, и качество модели по МАЕ выше (значение функции ошибки ниже).